# Load Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install polars==1.25.2 >> _


In [3]:
!pip install implicit >> _

In [4]:
!mkdir data

Подпапка или файл data уже существует.


In [5]:
# # takes 5 minutes
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/clickstream.pq -O data/clickstream.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/test_users.pq -O data/test_users.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/cat_features.pq -O data/cat_features.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/text_features.pq -O data/text_features.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/events.pq -O data/events.pq >> _


In [6]:
from datetime import timedelta
import polars as pl
import implicit

In [26]:
DATA_DIR = 'data/'

df_test_users = pl.read_parquet(f'{DATA_DIR}/test_users.pq')
df_clickstream = pl.read_parquet(f'{DATA_DIR}/clickstream.pq')

df_cat_features = pl.read_parquet(f'{DATA_DIR}/cat_features.pq')
df_text_features = pl.read_parquet(f'{DATA_DIR}/text_features.pq')
df_event = pl.read_parquet(f'{DATA_DIR}/events.pq')

# PREPARE TRAIN EVAL

In [27]:
EVAL_DAYS_TRESHOLD = 14

In [28]:
# устанавливаем временной порог
treshhold = df_clickstream['event_date'].max() - timedelta(days=EVAL_DAYS_TRESHOLD)
treshhold

datetime.datetime(2025, 2, 9, 0, 0)

In [29]:
# поведение пользователя до этого порога - трейн. после - валидационная
df_train = df_clickstream.filter(df_clickstream['event_date']<= treshhold)
df_eval = df_clickstream.filter(df_clickstream['event_date']> treshhold)[['cookie', 'node', 'event']]
df_train.shape, df_eval.shape

((45631770, 7), (23174382, 3))

Трейн - активность каждого пользователя ранее 2х недель назад. Эвал - активность каждого пользователя в последние 2 недели (взаимодействие с группами).

In [30]:
# оставляем в df_eval данные по активности пользователей,
# которые не имеют соответствий в df_train по указанным ключам
# используя 'cookie' и 'node' (пользователя и группу товара) в качестве ключей
# anti - чтобы выбрать только те строки из df_eval, которых нет в df_train по cookie и node
df_eval = df_eval.join(df_train, on=['cookie', 'node'], how='anti')
df_eval

cookie,node,event
i64,u32,i64
1,196744,17
1,48631,17
1,267694,17
1,196909,17
1,402072,17
…,…,…
149993,195297,17
149993,195034,17
149995,115733,17


In [12]:
# оставляем в df_eval только те строки,
# где значение в столбце 'event' (событие клика) присутствует в уникальных событиях из df_event,
# которые соответствуют условию is_contact == 1
df_eval = df_eval.filter(
    pl.col('event').is_in(
        df_event.filter(pl.col('is_contact')==1)['event'].unique()
    )
)

In [31]:
df_event.filter(pl.col('is_contact')==1)['event'].unique()

event
i64
0
1
2
4
5
…
13
14
15


In [25]:
df_eval

cookie,node,event
i64,u32,i64
43996,334165,10
116178,198564,4
13684,214349,10
95493,214339,19
462,193486,10
…,…,…
142376,360255,10
88517,115822,10
21609,27839,13


In [13]:
# в df_eval оставляем только те строки,
# где значение в столбце 'cookie' и 'node' присутствует в уникальных значениях из df_train
# то есть осталвяем только пользователей и их группы, которые есть и в трейне, и в тесте.
# то есть оставляем только записи о клиентах, если 14 или более дней назад такой интерес был 

df_eval = df_eval.filter(
    pl.col('cookie').is_in(df_train['cookie'].unique())  
    # в df_eval оставляем строки, где 'cookie' (пользователи) есть в уникальных значениях 'cookie' из df_train
).filter(
    pl.col('node').is_in(df_train['node'].unique())  
    # в df_eval оставляем строки, где 'node' (интерес) есть в уникальных значениях 'node' из df_train
)

In [14]:
# оставляем уникальные комбинации пользователей и их групп
df_eval = df_eval.unique(['cookie', 'node'])

# TRAIN MODEL

## ALS

In [15]:
def get_als_pred(users, nodes, user_to_pred):
    # Получаем уникальные идентификаторы пользователей и узлов (объявлений)
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()
        
    # Создаем словари для сопоставления идентификаторов пользователей и узлов с их индексами
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v: k for k, v in item_id_to_index.items()}
    
    # Заменяем идентификаторы пользователей и узлов на их индексы
    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()
    
    # Создаем значение для разреженной матрицы (в данном случае все значения равны 1)
    values = [1] * len(users)
    
    # Создаем разреженную матрицу, представляющую взаимодействия пользователей и узлов
    sparse_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
    
    # Инициализируем модель ALS (Alternating Least Squares) с заданными параметрами
    model = implicit.als.AlternatingLeastSquares(iterations=10, factors=60)
    # Обучаем модель на разреженной матрице
    model.fit(sparse_matrix)
    
    # Получаем индексы пользователей, для которых нужно сделать рекомендации
    user4pred = np.array([user_id_to_index[i] for i in user_to_pred])
    
    # Генерируем рекомендации для указанных пользователей
    recommendations, scores = model.recommend(user4pred, sparse_matrix[user4pred], N=40, filter_already_liked_items=True)
    
    # Создаем DataFrame для хранения рекомендаций, идентификаторов узлов и оценок
    df_pred = pl.DataFrame(
        {
            'node': [
                [index_to_item_id[i] for i in i] for i in recommendations.tolist()  # Преобразуем индексы обратно в идентификаторы узлов
            ], 
            'cookie': list(user_to_pred),  # Сохраняем идентификаторы пользователей
            'scores': scores.tolist()  # Сохраняем оценки для каждой рекомендации
        }
    )
    
    # Разворачиваем DataFrame, чтобы каждая строка содержала одну рекомендацию
    df_pred = df_pred.explode(['node', 'scores'])
    
    return df_pred  # Возвращаем DataFrame с рекомендациями


In [16]:
from scipy.sparse import csr_matrix
import numpy as np
import implicit


users = df_train["cookie"]
nodes = df_train["node"]
eval_users = df_eval['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes,eval_users )

C:\Users\kantoshina\anaconda3\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

## popular

In [17]:
def get_popular(df):
    popukar_node = df.group_by('node').agg(pl.col('cookie').count()).sort('cookie').tail(40)['node'].to_list()
    df_pred_pop = pl.DataFrame({'node': [popukar_node for i in range(len(eval_users))], 'cookie': eval_users})
    df_pred_pop = df_pred_pop.explode('node')
    return df_pred_pop

train_pop = get_popular(df_train)

# CALC EVAL METRICS

In [18]:
def recall_at(df_true, df_pred, k=40):
    return  df_true[['node', 'cookie']].join(
        df_pred.group_by('cookie').head(k).with_columns(value=1)[['node', 'cookie', 'value']], 
        how='left',
        on = ['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        [
            pl.col('value').sum()/pl.col(
                'value'
            ).count()
        ]
    )['value'].mean()


In [19]:
recall_at(df_eval, df_pred, k=40)

0.15130550510137222

In [20]:
recall_at(df_eval, train_pop, k=40)

0.058067308552970216

# SUMBIT

In [21]:
users = df_clickstream["cookie"]
nodes = df_clickstream["node"]
test_users = df_test_users['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes, test_users )


  0%|          | 0/10 [00:00<?, ?it/s]

In [22]:
df_pred.write_csv('prediction.csv')

In [23]:
df_pred

node,cookie,scores
i64,i64,f64
1907,1,1.20856
30443,1,1.203187
214377,1,1.156842
214339,1,1.116149
152705,1,1.105107
…,…,…
117348,149999,0.501848
120273,149999,0.500865
117342,149999,0.500647


In [ ]:
import polars as pl

def recall_at(df_solution: pl.DataFrame, df_pred: pl.DataFrame, k=40):
    assert df_pred.group_by(['cookie']).agg(pl.col('node').count())['node'].max() <41 , 'send more then 40 nodes per cookie'
    assert 'node' in df_pred.columns, 'node columns does not exist'
    assert 'cookie' in df_pred.columns, 'cookie columns does not exist'
    assert df_pred.with_columns(v = 1).group_by(['cookie','node']).agg(pl.col('v').count())['v'].max() == 1 , 'more then 1 cookie-node pair'
    assert df_pred['cookie'].dtype == pl.Int64, 'cookie must be int64'
    assert df_pred['node'].dtype == pl.Int64, 'node must be int64'
    
    return  df_solution[['node', 'cookie']].join(
        df_pred.group_by('cookie').head(k).with_columns(value=1)[['node', 'cookie', 'value']], 
        how='left',
        on = ['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        [
            pl.col('value').sum()/pl.col(
                'value'
            ).count()
        ]
    )['value'].mean()

def main(solution_path: str, prediction_path: str, stage: int):
    return recall_at(pl.read_csv(solution_path).filter(stage=stage), pl.read_csv(prediction_path))